In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openpyxl

base_url = 'http://quotes.toscrape.com'
url = base_url

data = []

### Crawl the first page

In [7]:
base_url = 'http://quotes.toscrape.com'
url = base_url

data = []

soup = BeautifulSoup(requests.get(url).text)
for e in soup.select('div.quote'):
    # Save basic information of authors
    qoute = {
        'author':e.select_one('small.author').text,
        'qoute':e.select_one('span.text').text
    }
    data.append(qoute)

### Crawl the first page with author's deeper details

In [28]:
base_url = 'http://quotes.toscrape.com'
url = base_url

data = []

# Function to get the date of birth and place of birth in the subpage
def get_author(url):

    soup = BeautifulSoup(requests.get(url).text)
    author = {
        'dob': soup.find(class_='author-born-date').text,       
        'lob': soup.find(class_='author-born-location').text,
        'url': url
    }
    return author

try:
    response = requests.get(url).text 
except requests.exceptions.HTTPError as err:
    print(f'An HTTP error occurred: {err}')

try:
    soup = BeautifulSoup(response, 'html.parser')
except bs4.FeatureNotFound as err:
    print(f'An error occurred while parsing the HTML: {err}')

for e in soup.select('div.quote'):
    # Save basic information of authors
    qoute = {
        'author':e.find('small', class_='author').text,
        'qoute':e.find('span', class_='text').text
    }
    qoute.update(get_author(base_url+e.a.get('href')))
    data.append(qoute)
pd.DataFrame(data)

,author,qoute,dob,lob,url
0,Albert Einstein,“The world as we have created it is a process ...,"March 14, 1879","in Ulm, Germany",http://quotes.toscrape.com/author/Albert-Einstein
1,J.K. Rowling,"“It is our choices, Harry, that show what we t...","July 31, 1965","in Yate, South Gloucestershire, England, The U...",http://quotes.toscrape.com/author/J-K-Rowling
2,Albert Einstein,“There are only two ways to live your life. On...,"March 14, 1879","in Ulm, Germany",http://quotes.toscrape.com/author/Albert-Einstein
3,Jane Austen,"“The person, be it gentleman or lady, who has ...","December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",http://quotes.toscrape.com/author/Jane-Austen
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","June 01, 1926",in The United States,http://quotes.toscrape.com/author/Marilyn-Monroe
5,Albert Einstein,“Try not to become a man of success. Rather be...,"March 14, 1879","in Ulm, Germany",http://quotes.toscrape.com/author/Albert-Einstein
6,André Gide,“It is better to be hated for what you are tha...,"November 22, 1869","in Paris, France",http://quotes.toscrape.com/author/Andre-Gide
7,Thomas A. Edison,"“I have not failed. I've just found 10,000 way...","February 11, 1847","in Milan, Ohio, The United States",http://quotes.toscrape.com/author/Thomas-A-Edison
8,Eleanor Roosevelt,“A woman is like a tea bag; you never know how...,"October 11, 1884",in The United States,http://quotes.toscrape.com/author/Eleanor-Roos...
9,Steve Martin,"“A day without sunshine is like, you know, nig...","August 14, 1945","in Waco, Texas, The United States",http://quotes.toscrape.com/author/Steve-Martin


### Trying to crawl with pagination

In [26]:
base_url = 'http://quotes.toscrape.com'
url = base_url

# Repeats until not seeing the next page button
while True:
    try:
        response = requests.get(url).text 
    except requests.exceptions.HTTPError as err:
        print(f'An HTTP error occurred: {err}')

    try:
        soup = BeautifulSoup(response, 'html.parser')
    except bs4.FeatureNotFound as err:
        print(f'An error occurred while parsing the HTML: {err}')
    
    if soup.select_one('li.next a'):
        # Get URL to the next page
        url=base_url+soup.select_one('li.next a').get('href')
        print(url)
    else:
        break

http://quotes.toscrape.com/page/2/
http://quotes.toscrape.com/page/3/
http://quotes.toscrape.com/page/4/
http://quotes.toscrape.com/page/5/
http://quotes.toscrape.com/page/6/
http://quotes.toscrape.com/page/7/
http://quotes.toscrape.com/page/8/
http://quotes.toscrape.com/page/9/
http://quotes.toscrape.com/page/10/


### Crawl date with pagination

In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openpyxl

base_url = 'http://quotes.toscrape.com'
url = base_url

data = []

# Function to get the date of birth and place of birth in the subpage
def get_author(url):
    soup = BeautifulSoup(requests.get(url).text)
    author = {
        'date_of_birth': soup.find(class_='author-born-date').text,       
        'place_of_birth': soup.find(class_='author-born-location').text,
        'url': url
    }
    return author

# Repeats until not seeing the next page button
while True:
    try:
        response = requests.get(url).text 
    except requests.exceptions.HTTPError as err:
        print(f'An HTTP error occurred: {err}')

    try:
        soup = BeautifulSoup(response, 'html.parser')
    except bs4.FeatureNotFound as err:
        print(f'An error occurred while parsing the HTML: {err}')

    for e in soup.select('div.quote'):
        # Save basic information of authors
        qoute = {
            'author':e.find('small', class_='author').text,
            'qoute':e.find('span', class_='text').text
        }
        qoute.update(get_author(base_url+e.a.get('href')))
        data.append(qoute)

    if soup.select_one('li.next a'):
        # Get URL to the next page
        url=base_url+soup.select_one('li.next a').get('href')
    else:
        break
    
# Convert python dictionary to Pandas dataframe
df = pd.DataFrame(data)

# Save Pandas dataframe to .csv and .xlsx file
df.to_csv('../data/processed/data.cv')
df.to_excel('../data/processed/data.xlsx')